In [1]:
import numpy as np
import os

layerList = ["가", "나", "다", "라", "마", "바", "아", "자", "차", "카", "타", "파", "하"]

all_x_array = []
all_y_array = []

for dir in layerList:
    fileList = os.listdir(f"../dataset_dir/image_npy/{dir}")

    for file in fileList:
        y_array = dict.fromkeys(layerList, 0) # 12000개의 str 값을 0으로 초기화하는 dict 생성

        if file[-8:-4] == "data":
            x_array = np.load(f"../dataset_dir/image_npy/{dir}/{file}")
            all_x_array.append(x_array.tolist())

            y_array[dir] = 1 # 사용자가 지정한 str 값을 1로 변경
            y_array = list(y_array.values())
            all_y_array.append(y_array)


FileNotFoundError: [Errno 2] No such file or directory: '../dataset_dir/image_npy/가'

In [5]:
x_array = np.array(all_x_array)
y_array = np.array(all_y_array)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    x_array, y_array, test_size=0.4, shuffle=False, random_state=1004
)

In [7]:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [8]:
image_w, image_h = X_train.shape[1], X_train.shape[2]

X_train = X_train.reshape(-1, image_w, image_h, 1)
X_test = X_test.reshape(-1, image_w, image_h, 1)

In [9]:
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(476, 200, 200, 1) (318, 200, 200, 1)
(476, 13) (318, 13)


In [10]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus: # 텐서플로가 첫 번째 GPU만 사용하도록 제한
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    print(f"{gpus[0]} 해당 GPU가 할당 되었습니다.")
  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)

2024-02-25 06:01:08.979198: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-25 06:01:09.017811: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 06:01:09.017844: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 06:01:09.018883: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-25 06:01:09.024485: I tensorflow/core/platform/cpu_feature_guar

In [1]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

2024-02-25 06:02:08.446409: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-25 06:02:08.475260: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 06:02:08.475288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 06:02:08.476187: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-25 06:02:08.481161: I tensorflow/core/platform/cpu_feature_guar

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16379112695775948304
xla_global_id: -1
]


2024-02-25 06:02:10.327993: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: UNKNOWN ERROR (34)


In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [11]:
import tensorflow as tf
import keras

model = keras.Sequential([
    keras.Input(shape=(200, 200, 1)),
    # keras.layers.Dense(1250, activation='relu'),
    # keras.layers.Dense(625, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(512, activation='relu'),
    # keras.layers.Dense(156, activation='relu'),
    # keras.layers.Dense(78, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation='relu'),
    # keras.layers.Dense(19, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(32, activation='relu'),
    # keras.layers.Dense(19, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(13, activation='softmax'),
])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 200, 200, 1)       0         
                                                                 
 dense (Dense)               (None, 200, 200, 512)     1024      
                                                                 
 dropout_1 (Dropout)         (None, 200, 200, 512)     0         
                                                                 
 dense_1 (Dense)             (None, 200, 200, 64)      32832     
                                                                 
 dropout_2 (Dropout)         (None, 200, 200, 64)      0         
                                                                 
 dense_2 (Dense)             (None, 200, 200, 32)      2080      
                                                                 
 dropout_3 (Dropout)         (None, 200, 200, 32)      0

In [13]:
batch_size = 16
epochs = 5

model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(0.05), metrics=["accuracy"])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, Y_test))

Epoch 1/5
20/30 [===================>..........] - ETA: 18s - loss: 930.4255 - accuracy: 0.1406